इस आलेख में शामिल जप्टर नोटबुक [अध्याय 4 कोड बेस](https://github.com/datawhalechina/learn-nlp-with-transformers/tree/main/docs/%E7%AF%87%E7%) में हैं AB%A04-%E4%BD%BF%E7%94%A8Transformers%E8%A7%A3%E5%86%B3NLP%E4%BB%BB%E5%8A%A1).

यदि आप इस नोटबुक को Google Colab में खोल रहे हैं, तो आपको उन्हें स्थापित करने के लिए ट्रांसफॉर्मर और 🤗Datasets लाइब्रेरीज़ को इंस्टॉल करने की आवश्यकता हो सकती है।

In [ ]:
!pip install datasets transformers seqeval

यदि आप इस नोटबुक को स्थानीय रूप से खोल रहे हैं, तो कृपया सुनिश्चित करें कि आपने उपरोक्त निर्भरताएं स्थापित की हैं। आप इस नोटबुक का मल्टी-जीपीयू वितरित प्रशिक्षण संस्करण भी यहां पा सकते हैं (https://github.com/huggingface/transformers/tree/)। मास्टर/उदाहरण/टोकन-वर्गीकरण)।

इस अनुभाग में शामिल मॉडल संरचना मूल रूप से पिछले अध्याय में BERT के समान है। आपको विशिष्ट कार्यों के लिए डेटा प्रोसेसिंग विधियां और मॉडल प्रशिक्षण विधियां अतिरिक्त रूप से सीखने की आवश्यकता है।

# *अनुक्रम लेबलिंग (टोकन-स्तरीय वर्गीकरण समस्या)*

अनुक्रम लेबलिंग को टोकन-स्तरीय वर्गीकरण समस्या के रूप में भी देखा जा सकता है: प्रत्येक टोकन को वर्गीकृत करें, इस नोटबुक में, हम दिखाएंगे कि [🤗 ट्रांसफॉर्मर] (https://github.com/huggingface/transformers) में ट्रांसफार्मर मॉडल का उपयोग कैसे करें। टोकन-स्तरीय वर्गीकरण करें। टोकन-स्तरीय वर्गीकरण कार्य आमतौर पर पाठ में प्रत्येक टोकन के लिए एक लेबल परिणाम की भविष्यवाणी करने का संदर्भ देते हैं। नीचे दिया गया चित्र एक एनईआर इकाई संज्ञा पहचान कार्य दिखाता है।

![एनईआर कार्य का प्रतिनिधित्व करने वाला विजेट अनुमान](https://github.com/huggingface/notebooks/blob/master/examples/images/token_classification.png?raw=1)

सबसे आम टोकन-स्तरीय वर्गीकरण कार्य:

- एनईआर (नामित-इकाई पहचान) पाठ में संज्ञाओं और संस्थाओं को अलग करता है (व्यक्ति के नाम, संगठन के नाम, स्थान के नाम...)।

- पीओएस (पार्ट-ऑफ-स्पीच टैगिंग (पार्ट-ऑफ-स्पीच टैगिंग) टोकन को उनके व्याकरण (संज्ञा, क्रिया, विशेषण, आदि) के अनुसार टैग करता है।
- चंकिंग (चंकिंग वाक्यांश) एक ही वाक्यांश के टोकन को एक साथ रखता है।

उपरोक्त कार्यों के लिए, हम दिखाएंगे कि डी को कैसे लोड किया जाएएटासेट एक साधारण डेटासेट लाइब्रेरी का उपयोग करता है और ट्रांसफार्मर में `ट्रेनर` इंटरफ़ेस का उपयोग करके पूर्व-प्रशिक्षित मॉडल को ठीक करता है।

जब तक पूर्व-प्रशिक्षित ट्रांसफार्मर मॉडल में शीर्ष पर एक टोकन वर्गीकरण तंत्रिका नेटवर्क परत होती है (जैसे कि पिछले अध्याय में उल्लिखित 'बर्टफॉरटोकनक्लासिफिकेशन') (इसके अलावा, ट्रांसफार्मर लाइब्रेरी की नई टोकननाइज़र सुविधा के कारण, संबंधित पूर्व -प्रशिक्षित मॉडल को तेज़ टोकननाइज़र फ़ंक्शन की भी आवश्यकता हो सकती है, [यह तालिका] (https://huggingface.co/transformers/index.html#bigtable) देखें), फिर यह नोटबुक सैद्धांतिक रूप से विभिन्न प्रकार के ट्रांसफार्मर मॉडल का उपयोग कर सकता है ( [एमओडेल पैनल](https://huggingface.co/models)) किसी भी टोकन-स्तरीय वर्गीकरण कार्य को हल करने के लिए।

यदि आप जिस कार्य से निपट रहे हैं वह अलग है, तो संभावना है कि इसे संसाधित करने के लिए इस नोटबुक का उपयोग करने के लिए केवल मामूली बदलाव की आवश्यकता है, साथ ही, आपको अपने जीपीयू वीडियो मेमोरी के अनुसार फाइन-ट्यूनिंग प्रशिक्षण के लिए आवश्यक btach आकार को समायोजित करना चाहिए वीडियो मेमोरी अतिप्रवाह से बचने के लिए.

In [ ]:
task = "ner" #需要是"ner", "pos" 或者 "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## डेटा डाउनलोड करें

हम डेटा और संबंधित मेट्रिक्स को लोड करने के लिए [🤗 डेटासेट] (https://github.com/huggingface/datasets) लाइब्रेरी का उपयोग करेंगे। डेटा लोडिंग और मीट्रिक लोडिंग के लिए केवल सरल `load_dataset` और `load_metric` की आवश्यकता होती है।

In [ ]:
from datasets import load_dataset, load_metric

इस नोटबुक के उदाहरण [CONLL 2003 डेटासेट](https://www.aclweb.org/anthology/W03-0419.pdf) डेटासेट का उपयोग करते हैं। इस नोटबुक को डेटासेट लाइब्रेरी में किसी भी टोकन वर्गीकरण कार्य के लिए काम करना चाहिए एक json/csv फ़ाइल से अपने स्वयं के कस्टम डेटासेट का उपयोग कर रहे हैं, आपको यह जानने के लिए [डेटासेट दस्तावेज़] (https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) की जांच करनी होगी इसे लोड करने के लिए कस्टम डेटासेट को लोडिंग प्रॉपर्टी नामों में कुछ समायोजन की आवश्यकता हो सकती है।

In [ ]:
datasets = load_dataset("conll2003")

`डेटासेट` ऑब्जेक्ट स्वयं प्रशिक्षण सेट, सत्यापन सेट और परीक्षण सेट के लिए एक [`डेटासेटडिक्ट`] (https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict) डेटा संरचना है संबंधित डेटा प्राप्त करने के लिए संबंधित कुंजी (ट्रेन, सत्यापन, परीक्षण) का उपयोग करें।

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

चाहे प्रशिक्षण सेट, सत्यापन सेट या परीक्षण सेट में, डेटासेट में टोकन नामक एक कॉलम होता है (आम तौर पर बोलते हुए, पाठ को कई शब्दों में विभाजित किया जाता है) और लेबल नामक एक कॉलम होता है, जो टोकन के एनोटेशन से मेल खाता है।

डेटा विभाजन कुंजी (ट्रेन, सत्यापन, या परीक्षण) और एक सबस्क्रिप्ट दिए जाने पर, आप डेटा देख सकते हैं।

In [ ]:
datasets["train"][0]

{'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'id': '0',
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.']}

सभी डेटा लेबल को पूर्णांकों में एन्कोड किया गया है और इन्हें सीधे पूर्व-प्रशिक्षित ट्रांसफार्मर मॉडल द्वारा उपयोग किया जा सकता है। इन पूर्णांकों के एन्कोडिंग के अनुरूप वास्तविक श्रेणियां 'फीचर्स' में संग्रहीत की जाती हैं।

In [ ]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], names_file=None, id=None), length=-1, id=None)

इसलिए एनईआर को एक उदाहरण के रूप में लेते हुए, 0 से संबंधित लेबल श्रेणी "ओ" है, 1 "बी-पीईआर" से संबंधित है, और इसी तरह "ओ" का मतलब कोई विशेष इकाई नहीं है। इस उदाहरण में 4 इकाई श्रेणियां शामिल हैं (पीईआर, ओआरजी,)। LOC, MISC), जिनमें से प्रत्येक में एक B- (इकाई प्रारंभ टोकन) उपसर्ग और एक I- (इकाई मध्य टोकन) उपसर्ग है।

- व्यक्ति के लिए 'प्रति'
- संगठन के लिए 'ओआरजी'
- स्थान के लिए 'एलओसी'
- विविध के लिए 'MISC'

In [ ]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

यह समझने के लिए कि डेटा कैसा दिखता है, निम्न फ़ंक्शन यादृच्छिक रूप से डेटासेट से कुछ उदाहरणों का चयन करेगा और उन्हें प्रदर्शित करेगा।

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,id,tokens,pos_tags,chunk_tags,ner_tags
0,2227,"[Result, of, a, French, first, division, match, on, Friday, .]","[NN, IN, DT, JJ, JJ, NN, NN, IN, NNP, .]","[B-NP, B-PP, B-NP, I-NP, I-NP, I-NP, I-NP, B-PP, B-NP, O]","[O, O, O, B-MISC, O, O, O, O, O, O]"
1,2615,"[Mid-tier, golds, up, in, heavy, trading, .]","[NN, NNS, IN, IN, JJ, NN, .]","[B-NP, I-NP, B-PP, B-PP, B-NP, I-NP, O]","[O, O, O, O, O, O, O]"
2,10256,"[Neagle, (, 14-6, ), beat, the, Braves, for, the, third, time, this, season, ,, allowing, two, runs, and, six, hits, in, eight, innings, .]","[NNP, (, CD, ), VB, DT, NNPS, IN, DT, JJ, NN, DT, NN, ,, VBG, CD, NNS, CC, CD, NNS, IN, CD, NN, .]","[B-NP, O, B-NP, O, B-VP, B-NP, I-NP, B-PP, B-NP, I-NP, I-NP, B-NP, I-NP, O, B-VP, B-NP, I-NP, O, B-NP, I-NP, B-PP, B-NP, I-NP, O]","[B-PER, O, O, O, O, O, B-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,10720,"[Hansa, Rostock, 4, 1, 2, 1, 5, 4, 5]","[NNP, NNP, CD, CD, CD, CD, CD, CD, CD]","[B-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP]","[B-ORG, I-ORG, O, O, O, O, O, O, O]"
4,7125,"[MONTREAL, 70, 59, .543, 11]","[NNP, CD, CD, CD, CD]","[B-NP, I-NP, I-NP, I-NP, I-NP]","[B-ORG, O, O, O, O]"
5,3316,"[Softbank, Corp, said, on, Friday, that, it, would, procure, $, 900, million, through, the, foreign, exchange, market, by, September, 5, as, part, of, its, acquisition, of, U.S., firm, ,, Kingston, Technology, Co, .]","[NNP, NNP, VBD, IN, NNP, IN, PRP, MD, NN, $, CD, CD, IN, DT, JJ, NN, NN, IN, NNP, CD, IN, NN, IN, PRP$, NN, IN, NNP, NN, ,, NNP, NNP, NNP, .]","[B-NP, I-NP, B-VP, B-PP, B-NP, B-SBAR, B-NP, B-VP, B-NP, I-NP, I-NP, I-NP, B-PP, B-NP, I-NP, I-NP, I-NP, B-PP, B-NP, I-NP, B-PP, B-NP, B-PP, B-NP, I-NP, B-PP, B-NP, I-NP, O, B-NP, I-NP, I-NP, O]","[B-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-LOC, O, O, B-ORG, I-ORG, I-ORG, O]"
6,3923,"[Ghent, 3, Aalst, 2]","[NN, CD, NNP, CD]","[B-NP, I-NP, I-NP, I-NP]","[B-ORG, O, B-ORG, O]"
7,2776,"[The, separatists, ,, who, swept, into, Grozny, on, August, 6, ,, still, control, large, areas, of, the, centre, of, town, ,, and, Russian, soldiers, are, based, at, checkpoints, on, the, approach, roads, .]","[DT, NNS, ,, WP, VBD, IN, NNP, IN, NNP, CD, ,, RB, VBP, JJ, NNS, IN, DT, NN, IN, NN, ,, CC, JJ, NNS, VBP, VBN, IN, NNS, IN, DT, NN, NNS, .]","[B-NP, I-NP, O, B-NP, B-VP, B-PP, B-NP, B-PP, B-NP, I-NP, O, B-ADVP, B-VP, B-NP, I-NP, B-PP, B-NP, I-NP, B-PP, B-NP, O, O, B-NP, I-NP, B-VP, I-VP, B-PP, B-NP, B-PP, B-NP, I-NP, I-NP, O]","[O, O, O, O, O, O, B-LOC, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-MISC, O, O, O, O, O, O, O, O, O, O]"
8,1178,"[Doctor, Masserigne, Ndiaye, said, medical, staff, were, overwhelmed, with, work, ., ""]","[NNP, NNP, NNP, VBD, JJ, NN, VBD, VBN, IN, NN, ., ""]","[B-NP, I-NP, I-NP, B-VP, B-NP, I-NP, B-VP, I-VP, B-PP, B-NP, O, O]","[O, B-PER, I-PER, O, O, O, O, O, O, O, O, O]"
9,10988,"[Reuters, historical, calendar, -, September, 4, .]","[NNP, JJ, NN, :, NNP, CD, .]","[B-NP, I-NP, I-NP, O, B-NP, I-NP, O]","[B-ORG, O, O, O, O, O, O]"


## प्रीप्रोसेसिंग डेटा

मॉडल में डेटा फीड करने से पहले, हमें डेटा को प्रीप्रोसेस करना होगा। प्रीप्रोसेसिंग टूल को 'टोकनाइज़र' कहा जाता है। पहले इनपुट को टोकनाइज़ करता है, फिर टोकन को प्री-मॉडल में आवश्यक संबंधित टोकन आईडी में परिवर्तित करता है। उन्हें मॉडल द्वारा आवश्यक इनपुट प्रारूप में परिवर्तित करता है।

डेटा प्रीप्रोसेसिंग के उद्देश्य को प्राप्त करने के लिए, हम अपने टोकननाइज़र को इंस्टेंट करने के लिए `AutoTokenizer.from_pretrained` विधि का उपयोग करते हैं, जो सुनिश्चित करता है:

- हमें एक टोकननाइज़र मिलता है जो टी से मेल खाता हैo पूर्व-प्रशिक्षित मॉडल को एक-एक करके।
- निर्दिष्ट मॉडल चेकपॉइंट के अनुरूप टोकननाइज़र का उपयोग करते समय, हम मॉडल के लिए आवश्यक शब्दावली, अधिक सटीक रूप से, टोकन शब्दावली भी डाउनलोड करते हैं।

इस डाउनलोड की गई टोकन शब्दावली को कैश कर दिया जाएगा ताकि दोबारा उपयोग करने पर इसे दोबारा डाउनलोड न किया जा सके।

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

ध्यान दें: निम्नलिखित कोड के लिए आवश्यक है कि टोकननाइज़र ट्रांसफार्मर प्रकार का होना चाहिए। PreTrainedTokenizerFast, क्योंकि हमें प्रीप्रोसेसिंग के दौरान फास्ट टोकननाइज़र (जैसे मल्टी-थ्रेडेड फास्ट टोकननाइज़र) की कुछ विशेष सुविधाओं का उपयोग करने की आवश्यकता होती है।

मॉडल के अनुरूप लगभग सभी टोकननाइज़र में तेज़ टोकननाइज़र होते हैं। हम [मॉडल टोकनाइज़र कॉरेस्पोंडेंस टेबल] (https://huggingface.co/transformers/index.html#bigta) में सभी पूर्व-प्रशिक्षित मॉडल के अनुरूप टोकननाइज़र की विशेषताएं देख सकते हैं।ब्ली)।

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

यह देखने के लिए कि क्या मॉडल में तेज़ टोकननाइज़र है, [यहां मॉडल की बड़ी तालिका] (https://huggingface.co/transformers/index.html#bigtable) देखें।

टोकननाइज़र एकल टेक्स्ट या टेक्स्ट की एक जोड़ी को प्रीप्रोसेस कर सकता है। टोकननाइज़र प्रीप्रोसेसिंग के बाद प्राप्त डेटा पूर्व-प्रशिक्षित मॉडल के इनपुट प्रारूप से मिलता है।

In [ ]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 3975, 2046, 2616, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

ध्यान दें कि ट्रांसफार्मर पूर्व-प्रशिक्षित मॉडल आमतौर पर पूर्व-प्रशिक्षण के दौरान उपशब्दों का उपयोग करते हैं। यदि हमारे टेक्स्ट इनपुट को शब्दों में विभाजित किया गया है, तो इन शब्दों को हमारे टोकननाइज़र द्वारा आगे विभाजित किया जाएगा।

In [ ]:
example = datasets["train"][4]
print(example["tokens"])

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']


In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'germany', "'", 's', 'representative', 'to', 'the', 'european', 'union', "'", 's', 'veterinary', 'committee', 'werner', 'z', '##wing', '##mann', 'said', 'on', 'wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']


शब्द "ज़्विंगमैन" और "शीपमीट" को आगे 3 उपसंकेतों में विभाजित किया गया है।

चूंकि एनोटेट किए गए डेटा को आमतौर पर शब्द स्तर पर एनोटेट किया जाता है, क्योंकि शब्द को उपटोकन में भी विभाजित किया जाता है, इसका मतलब है कि हमें पूर्व-प्रशिक्षित आवश्यकताओं के कारण, एनोटेट किए गए डेटा के उपटोकन को भी संरेखित करने की आवश्यकता है मॉडल इनपुट प्रारूप में, कुछ विशेष प्रतीकों जैसे: `[CLS]` और `[SEP]` की अक्सर आवश्यकता होती है।

In [ ]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(31, 39)

टोकननाइज़र के पास एक ``word_ids`` विधि है जो इसमें हमारी सहायता कर सकती है।

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]


हम देख सकते हैं कि वर्ड_आईडी प्रत्येक सबटोकन स्थिति को एक शब्द सबस्क्रिप्ट में मैप करता है, पहली स्थिति 0 वें शब्द से मेल खाती है, और दूसरी और तीसरी स्थिति पहले शब्द से मेल खाती है, इस सूची के साथ, हम कर सकते हैं उपटोकन को शब्दों और लेबलों के साथ संरेखित करें।

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

39 39


हम आमतौर पर मॉडल में विशेष वर्णों का लेबल -100 पर सेट करते हैं, -100 को आमतौर पर नजरअंदाज कर दिया जाता है और नुकसान की गणना नहीं की जाती है।

हमारे पास लेबल संरेखित करने के दो तरीके हैं:
- एक शब्द और एक लेबल में एकाधिक उपटोकन संरेखित करें
- एकाधिक उपटोकन के पहले उपटोकन को एक शब्द और एक लेबल में संरेखित करें, और अन्य उपटोकन को सीधे -100 असाइन करें।

हम ये दो विधियाँ प्रदान करते हैं, जिन्हें `label_all_tokens = True` द्वारा स्विच किया जा सकता है।

In [ ]:
label_all_tokens = True

अंत में हमने सब कुछ एक साथ अपने प्रीप्रोसेसिंग फ़ंक्शन में डाल दिया। `is_split_into_words=True` ऊपर पहले ही समाप्त हो चुका है।

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
# विशेष टोकन में एक शब्द आईडी होती है जो कि कोई नहीं है, हम लेबल को -100 पर सेट करते हैं ताकि वे स्वचालित रूप से हों
# हानि फ़ंक्शन में अनदेखा किया गया।
            if word_idx is None:
                label_ids.append(-100)
# हम प्रत्येक शब्द के पहले टोकन के लिए लेबल सेट करते हैं।
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
# किसी शब्द में अन्य टोकन के लिए, हम लेबल को वर्तमान लेबल या -100 पर निर्भर करते हुए सेट करते हैं
# लेबल_ऑल_टोकन ध्वज।
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

उपरोक्त प्रीप्रोसेसिंग फ़ंक्शन एक नमूने या एकाधिक नमूना उदाहरणों को संसाधित कर सकता है। यदि यह एकाधिक नमूनों को संसाधित करता है, तो यह एकाधिक नमूनों के प्रीप्रोसेसिंग के परिणामों की एक सूची लौटाता है।

In [ ]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], [101, 2848, 13934, 102], [101, 9371, 2727, 1011, 5511, 1011, 2570, 102], [101, 1996, 2647, 3222, 2056, 2006, 9432, 2009, 18335, 2007, 2446, 6040, 2000, 10390, 2000, 18454, 2078, 2329, 12559, 2127, 6529, 5646, 3251, 5506, 11190, 4295, 2064, 2022, 11860, 2000, 8351, 1012, 102], [101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100], [-100, 1, 2, -100], [-100, 5, 0, 

इसके बाद, डेटासेट डेटासेट में सभी नमूनों को प्रीप्रोसेसिंग फ़ंक्शन तैयार_ट्रेन_फीचर्स को सभी नमूनों पर लागू करने के लिए मैप फ़ंक्शन का उपयोग करके प्रीप्रोसेस किया जाता है।

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

इससे भी बेहतर, अगली बार संसाधित होने पर पुनर्गणना से बचने के लिए लौटाए गए परिणाम स्वचालित रूप से कैश किए जाते हैं (लेकिन ध्यान रखें कि यदि इनपुट बदलता है, तो यह कैश से प्रभावित हो सकता है!) डेटासेट लाइब्रेरी फ़ंक्शन यह निर्धारित करने के लिए इनपुट पैरामीटर का पता लगाएगा यदि कोई परिवर्तन नहीं हैं, तो कैश्ड डेटा का उपयोग किया जाएगा। यदि परिवर्तन हैं, तो डेटा को पुन: संसाधित किया जाएगा। हालांकि, यदि इनपुट पैरामीटर नहीं बदलते हैं, तो जब आप चाहें तो कैश को साफ़ करना सबसे अच्छा है चांगई इनपुट। इसे साफ़ करने का तरीका `load_from_cache_file=False` पैरामीटर का उपयोग करना है। इसके अलावा, ऊपर इस्तेमाल किया गया `batched=True` पैरामीटर टोकननाइज़र की एक विशेषता है, क्योंकि यह समानांतर में इनपुट को संसाधित करने के लिए कई थ्रेड्स का उपयोग करता है। .

## पूर्व-प्रशिक्षित मॉडल को फाइन-ट्यून करें

अब जब डेटा तैयार है, तो हमें अपने पूर्व-प्रशिक्षित मॉडल को डाउनलोड और लोड करना होगा, और फिर पूर्व-प्रशिक्षित मॉडल को फाइन-ट्यून करना होगा क्योंकि हम एक seq2seq कार्य कर रहे हैं, हमें एक मॉडल वर्ग की आवश्यकता है जो इस कार्य को हल कर सके क्लास `AutoModelForTokenClassification` का उपयोग करें। टोकननाइज़र के समान, `from_pretrained` विधि भी हमें मॉडल को डाउनलोड करने और लोड करने में मदद कर सकती है, और यह मॉडल को कैश भी करेगी ताकि हम मॉडल को बार-बार डाउनलोड न करें।

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN t

चूंकि हमारा फाइन-ट्यूनिंग कार्य टोकन वर्गीकरण कार्य है, और हमने एक पूर्व-प्रशिक्षित भाषा मॉडल लोड किया है, हमें संकेत दिया जाएगा कि मॉडल लोड करते समय कुछ बेमेल तंत्रिका नेटवर्क पैरामीटर हटा दिए गए थे (उदाहरण के लिए, पूर्व का तंत्रिका नेटवर्क प्रमुख) -प्रशिक्षित भाषा मॉडल को फेंक दिया गया था, और टोकन वर्गीकरण के तंत्रिका नेटवर्क प्रमुख को यादृच्छिक रूप से प्रारंभ किया गया था)।

`ट्रेनर` प्रशिक्षण उपकरण प्राप्त करने के लिए, हमें 3 और तत्वों की आवश्यकता है, जिनमें से सबसे महत्वपूर्ण है प्रशिक्षण सेटिंग्स/पैरामीटर [`ट्रेनिंगआर्ग्यूमेंट्स`](https://huggingface.co/transformers/main_classes/trainer.html# ट्रांसफार्मर.ट्रेनिंगआर्ग्युमेंट्स) इस प्रशिक्षण सेटिंग में वे सभी गुण शामिल हैं जो प्रशिक्षण प्रक्रिया को परिभाषित कर सकते हैं।

In [ ]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

उपरोक्त मूल्यांकन_रणनीति = "युग" पैरामीटर प्रशिक्षण कोड को बताता है कि हम प्रति युग एक बार सत्यापन मूल्यांकन करेंगे।

इस नोटबुक से पहले बैच_आकार को ऊपर परिभाषित किया गया है।

अंत में, हमें अपने संसाधित इनपुट को मॉडल में फीड करने के लिए एक डेटा कोलेटर की आवश्यकता होती है।

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

`ट्रेनर` को स्थापित करने के लिए आखिरी चीज मूल्यांकन पद्धति को परिभाषित करना है। हम मूल्यांकन भेजने से पहले [`seqeval`](https://github.com/chalki-works/seqeval) मीट्रिक का उपयोग करते हैं मूल्यांकन के लिए मॉडल भविष्यवाणियों, हम कुछ डेटा पोस्ट-प्रोसेसिंग भी करेंगे:

In [ ]:
metric = load_metric("seqeval")

मूल्यांकन का इनपुट पूर्वानुमानों और लेबलों की एक सूची है

In [ ]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'LOC': {'f1': 1.0, 'number': 2, 'precision': 1.0, 'recall': 1.0},
 'ORG': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'PER': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0}

मॉडल पूर्वानुमान परिणामों पर कुछ पोस्ट-प्रोसेसिंग करें:
- पूर्वानुमानित वर्गीकरण की अधिकतम संभावना वाली सबस्क्रिप्ट का चयन करें
-सबस्क्रिप्ट को लेबल में बदलें
--100 पर ध्यान न दें

निम्नलिखित फ़ंक्शन उपरोक्त चरणों को जोड़ता है।

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

# उपेक्षित सूचकांक (विशेष टोकन) हटाएं
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

हम सभी श्रेणियों की कुल परिशुद्धता/रिकॉल/एफ1 की गणना करते हैं, इसलिए हम एक श्रेणी की परिशुद्धता/रिकॉल/एफ1 को हटा देंगे

डेटा/मॉडल/पैरामीटर को `ट्रेनर` में डालें

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

प्रशिक्षण शुरू करने के लिए `ट्रेन` विधि को कॉल करें

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.237721,0.068198,0.903148,0.921132,0.912051,0.979713
2,0.053160,0.059337,0.927697,0.932990,0.930336,0.983113
3,0.029850,0.059346,0.929267,0.939143,0.934179,0.984257


TrainOutput(global_step=2634, training_loss=0.08569671253227518)

हम अन्य डेटासेट का मूल्यांकन करने के लिए फिर से मूल्यांकन पद्धति का उपयोग कर सकते हैं।

In [ ]:
trainer.evaluate()

{'eval_loss': 0.05934586375951767,
 'eval_precision': 0.9292672127518264,
 'eval_recall': 0.9391430808815304,
 'eval_f1': 0.9341790463472988,
 'eval_accuracy': 0.9842565968195466,
 'epoch': 3.0}

यदि हम किसी एकल श्रेणी की सटीकता/रिकॉल/एफ1 प्राप्त करना चाहते हैं, तो हम परिणामों को सीधे उसी मूल्यांकन फ़ंक्शन में इनपुट कर सकते हैं:

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# उपेक्षित सूचकांक (विशेष टोकन) हटाएं
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'LOC': {'precision': 0.949718574108818,
  'recall': 0.966768525592055,
  'f1': 0.9581677077418134,
  'number': 2618},
 'MISC': {'precision': 0.8132387706855791,
  'recall': 0.8383428107229894,
  'f1': 0.8255999999999999,
  'number': 1231},
 'ORG': {'precision': 0.9055232558139535,
  'recall': 0.9090466926070039,
  'f1': 0.9072815533980583,
  'number': 2056},
 'PER': {'precision': 0.9759552042160737,
  'recall': 0.9765985497692815,
  'f1': 0.9762767710049424,
  'number': 3034},
 'overall_precision': 0.9292672127518264,
 'overall_recall': 0.9391430808815304,
 'overall_f1': 0.9341790463472988,
 'overall_accuracy': 0.9842565968195466}

अंत में, अपने मॉडल को [🤗 मॉडल हब](https://huggingface.co/models) पर अपलोड करना न भूलें (कैसे देखने के लिए [यहां](https://huggingface.co/transformers/model_sharing.html) पर क्लिक करें अपलोड करने के लिए) फिर आप इस नोटबुक की शुरुआत की तरह, मॉडल नाम का उपयोग करके सीधे अपने अपलोड किए गए मॉडल का उपयोग कर सकते हैं।